In [125]:
from postgre import postgre

In [126]:
#we create a connection to our database i don't need an username or a password because my postgre 
#is running on localhost.
connection=postgre('','','javierllorente')

In [127]:
#we execute the query and it is transform to a Dataframe.
dataframe=connection.postgre_to_dataframe('select * from tripadvisor_hotels')
dataframe.head(200)

,name,address,zip_code,city,number_of_rooms,stars,services,min_price,max_price,phone_number,created_at_insert
0,4C Bravo Murillo,"Calle Bravo Murillo 180, 28020 Madrid, España",28020.0,Madrid,92.0,,"[Wifi gratuito, Desayuno incluido, Aire acondi...",69.0,140.0,None,2017-10-29 23:18:15.309081
1,7 Islas Hotel,"Calle Valverde, 14, 28004 Madrid, España",28004.0,Madrid,63.0,"Hotel de 4,0 estrellas","[Wifi gratuito, Desayuno buffet, Servicio de h...",90.0,212.0,None,2017-10-29 23:18:15.309081
2,AC Hotel Aravaca,"Camino Zarzuela 3, 28023 Madrid, España",28023.0,Madrid,110.0,"Hotel de 4,0 estrellas","[Wifi gratuito, Aparcamiento, Aire acondiciona...",64.0,125.0,None,2017-10-29 23:18:15.309081
3,AC Hotel Atocha,"Calle Delicias 42, 28045 Madrid, España",28045.0,Madrid,160.0,"Hotel de 4,0 estrellas","[Wifi gratuito, Aparcamiento, Desayuno buffet,...",95.0,214.0,None,2017-10-29 23:18:15.309081
4,AC Hotel Carlton Madrid,"Paseo Delicias 26, 28045 Madrid, España",28045.0,Madrid,112.0,"Hotel de 4,0 estrellas","[Wifi gratuito, Aparcamiento, Desayuno buffet,...",84.0,204.0,None,2017-10-29 23:18:15.309081
5,AC Hotel Cuzco,"Paseo Castellana 133, 28046 Madrid, España",28046.0,Madrid,319.0,"Hotel de 4,0 estrellas","[Internet gratuito, Aparcamiento, Desayuno buf...",95.0,266.0,None,2017-10-29 23:18:15.309081
6,AC Hotel Los Vascos,"Calle Vascos 27, 28040 Madrid, España",28040.0,Madrid,48.0,"Hotel de 3,0 estrellas","[Wifi gratuito, Aparcamiento, Aire acondiciona...",83.0,136.0,None,2017-10-29 23:18:15.309081
7,AC Hotel Madrid Feria,"Via Poblados 3, 28033 Madrid, España",28033.0,Madrid,154.0,"Hotel de 4,0 estrellas","[Wifi gratuito, Aparcamiento, Desayuno buffet,...",61.0,181.0,None,2017-10-29 23:18:15.309081
8,AC Hotel Recoletos,"Calle Recoletos 18, 28001 Madrid, España",28001.0,Madrid,63.0,"Hotel de 4,0 estrellas","[Wifi gratuito, Aparcamiento, Desayuno buffet,...",146.0,258.0,None,2017-10-29 23:18:15.309081
9,"AC Palacio del Retiro, Autograph Collection","Calle Alfonso XII 14, 28014 Madrid, España",28014.0,Madrid,50.0,"Hotel de 5,0 estrellas","[Wifi gratuito, Aparcamiento, Desayuno buffet,...",203.0,424.0,None,2017-10-29 23:18:15.309081


In [128]:
def stars(x):
    #we transform the stars column into numbers.
    x=str(x)
    if '1' in x:
        return 1
    elif '2' in x:
        return 2
    elif '3' in x:
        return 3 
    elif '4' in x:
        return 4
    elif '5' in x:
        return 5
    
dataframe['numeric_stars']=dataframe['stars'].map(lambda x: stars(x))
dataframe['numeric_stars'].head(4)

0    NaN
1    4.0
2    4.0
3    4.0
Name: numeric_stars, dtype: float64

In [156]:
#we perform some functions over the data.
functions={'numeric_stars':['mean'],'name':['count'],
           'max_price':['mean'],'min_price':['mean'],'number_of_rooms':['mean']}
stars_per_city = dataframe.groupby(dataframe['city']).agg(functions).round(2)
#we filter the data.
#first we reset our dataframe to got two index labels instead of three.
stars_per_city_reset=stars_per_city.copy()
stars_per_city_reset.reset_index(col_level=0)
stars_per_city_reset.columns = stars_per_city_reset.columns.get_level_values(0)
#we only analize citys with more than 30 hotels.
stars_per_city_reset=stars_per_city_reset[stars_per_city_reset['name']>30]
stars_per_city_reset[1:]

,number_of_rooms,numeric_stars,min_price,max_price,name
city,,,,,
Barcelona,88.21,3.44,85.05,223.18,856
Benalmádena,184.21,3.46,58.88,155.97,39
Benidorm,125.15,3.29,61.39,151.30,47
Calella,156.74,3.07,41.67,155.40,39
Estepona,145.84,3.40,81.27,201.46,31
Fuengirola,175.59,3.30,51.44,118.44,35
Madrid,109.17,3.44,81.17,206.98,747
Marbella,120.11,3.53,95.27,240.62,47
Málaga,80.10,3.18,68.58,154.38,82


In [130]:
#K-Means, clustering.
from sklearn.cluster import KMeans
Y=[i for i in stars_per_city_reset.index[1:]]
X=[["{0:.2f}".format(x1),"{0:.2f}".format(x2),"{0:.2f}".format(x3)] 
   for x1,x2,x3 in zip(stars_per_city_reset['numeric_stars'][1:],stars_per_city_reset['max_price'][1:],
                       stars_per_city_reset['number_of_rooms'])]
#cluster the data into three clusters,
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
y_pred = kmeans.predict(X)
# cluster result for each city.
cluster_dict={city:cluster for city,cluster in zip(Y,y_pred)}
cluster_dict

{'Barcelona': 2,
 'Benalmádena': 1,
 'Benidorm': 0,
 'Calella': 0,
 'Estepona': 2,
 'Fuengirola': 0,
 'Madrid': 2,
 'Marbella': 2,
 'Málaga': 0,
 'París': 2,
 'Ronda': 1,
 'Santiago': 1,
 'Sanxenxo': 1,
 'Sevilla': 1,
 'Torremolinos': 1,
 'Valencia': 2,
 'Vigo': 0}

In [131]:
#Agglomerative clustering.
from sklearn.cluster import AgglomerativeClustering
Y=[i for i in stars_per_city_reset.index[1:]]
X=[["{0:.2f}".format(x1),"{0:.2f}".format(x2),"{0:.2f}".format(x3)] 
   for x1,x2,x3 in zip(stars_per_city_reset['numeric_stars'][1:],stars_per_city_reset['max_price'][1:],
                       stars_per_city_reset['number_of_rooms'][1:])]
#cluster the data into three clusters.
agglomerative = AgglomerativeClustering(n_clusters=3)
y_pred=agglomerative.fit_predict(X)
# cluster result for each city.
cluster_dict={city:cluster for city,cluster in zip(Y,y_pred)}
cluster_dict

{'Barcelona': 0,
 'Benalmádena': 1,
 'Benidorm': 0,
 'Calella': 1,
 'Estepona': 0,
 'Fuengirola': 1,
 'Madrid': 0,
 'Marbella': 0,
 'Málaga': 0,
 'París': 0,
 'Ronda': 0,
 'Santiago': 2,
 'Sanxenxo': 2,
 'Sevilla': 0,
 'Torremolinos': 1,
 'Valencia': 0,
 'Vigo': 2}

In [132]:
#DBSCAN clustering.
from sklearn.cluster import DBSCAN
Y=[i for i in stars_per_city_reset.index[1:]]
X=[["{0:.2f}".format(x1),"{0:.2f}".format(x2),"{0:.2f}".format(x2)] 
   for x1,x2,x3 in zip(stars_per_city_reset['numeric_stars'][1:],stars_per_city_reset['max_price'][1:],
                       stars_per_city_reset['number_of_rooms'][1:])]
#In this case DBSCAN algorithim will set up the number of clusters automatically.
dbscan = DBSCAN()
y_pred = dbscan.fit_predict(X)
# cluster result for each city.
cluster_dict={city:cluster for city,cluster in zip(Y,y_pred)}
cluster_dict
#this algorithim does not work well with this dataset becuase it is really small.

{'Barcelona': -1,
 'Benalmádena': -1,
 'Benidorm': -1,
 'Calella': -1,
 'Estepona': -1,
 'Fuengirola': -1,
 'Madrid': -1,
 'Marbella': -1,
 'Málaga': -1,
 'París': -1,
 'Ronda': -1,
 'Santiago': -1,
 'Sanxenxo': -1,
 'Sevilla': -1,
 'Torremolinos': -1,
 'Valencia': -1,
 'Vigo': -1}

In [155]:
#DBSCAN clustering.
from sklearn.cluster import DBSCAN
Y=[i for i in stars_per_city_reset.index[1:]]
X=[["{0:.2f}".format(x1),"{0:.2f}".format(x2),"{0:.2f}".format(x3)] 
   for x1,x2,x3 in zip(stars_per_city_reset['numeric_stars'][1:],stars_per_city_reset['max_price'][1:],
                       stars_per_city_reset['number_of_rooms'][1:])]
#In this case DBSCAN algorithim will set up the number of clusters automatically.
dbscan = DBSCAN(min_samples=3, eps=18.0)
y_pred = dbscan.fit_predict(X)
# cluster result for each city.
cluster_dict={city:cluster for city,cluster in zip(Y,y_pred)}
cluster_dict
#we adapt the parameters to the alogorithm min_samples=the minimum number of cities to create a cluster.
#eps=max distance between the points.

{'Barcelona': -1,
 'Benalmádena': -1,
 'Benidorm': -1,
 'Calella': -1,
 'Estepona': -1,
 'Fuengirola': -1,
 'Madrid': -1,
 'Marbella': -1,
 'Málaga': -1,
 'París': -1,
 'Ronda': -1,
 'Santiago': 0,
 'Sanxenxo': 0,
 'Sevilla': -1,
 'Torremolinos': -1,
 'Valencia': -1,
 'Vigo': 0}